In [ ]:
# walkthrough capsule em
# 

In [ ]:
from datasets import mnist
reload(mnist)

In [ ]:
images, labels = mnist.load_mnist('data/mnist')
print('images shape:', images.shape)
print('labels shape:', labels.shape)

In [ ]:
from matplotlib import pyplot as plt

def inspect_mnist(index, images, labels):
  plt.title('Index: {:d}, Label: {:d}'.format(
    index, labels[index].argmax()
  ))
  plt.imshow(images[index][:, :, 0], cmap='gray')
  plt.show()

In [ ]:
inspect_mnist(46, images, labels)

In [1]:
import numpy as np
import tensorflow as tf

batch_size = 17

In [ ]:
x = tf.convert_to_tensor(images[:batch_size], dtype=tf.float32)
x

In [ ]:
y = tf.convert_to_tensor(labels[:batch_size], dtype=tf.uint8)
y

In [ ]:
# input -> conv1
conv1 = tf.contrib.layers.conv2d(
    x, num_outputs=32, kernel_size=[5, 5], stride=2, padding='VALID'
)
conv1

In [6]:
graph = tf.Graph()
sess = tf.InteractiveSession(graph=graph)

In [7]:
# aa = tf.constant(
#     np.arange(1, 13, dtype=np.int32), shape=[2, 2, 3]
# )
aa = tf.convert_to_tensor(
    np.arange(4 * 4 * 3).reshape((1, 4, 4, 3)), dtype=tf.float32
)
av = sess.run(aa)
av[:, 0:2, 0:2, :]

array([[[[  0.,   1.,   2.],
         [  3.,   4.,   5.]],

        [[ 12.,  13.,  14.],
         [ 15.,  16.,  17.]]]], dtype=float32)

In [8]:
ww = tf.convert_to_tensor(
    np.arange(2*2*3*5).reshape((2, 2, 3, 5)), dtype=tf.float32
)
wv = sess.run(ww)
wv[:, :, :, 0]

array([[[  0.,   5.,  10.],
        [ 15.,  20.,  25.]],

       [[ 30.,  35.,  40.],
        [ 45.,  50.,  55.]]], dtype=float32)

In [9]:
conv1 = tf.nn.conv2d(
    aa, filter=ww, strides=[1, 1, 1, 1], padding='VALID', name='conv1'
)
conv1

<tf.Tensor 'conv1:0' shape=(1, 3, 3, 5) dtype=float32>

In [10]:
conv1v = sess.run(conv1)
conv1v.shape

(1, 3, 3, 5)

In [11]:
np.sum(av[0, 0:2, 0:2, :] * wv[:, :, :, 0])

4060.0

In [2]:
def _get_variable_wrapper(
        name, shape=None, dtype=None, initializer=None,
        regularizer=None,
        trainable=True,
        collections=None,
        caching_device=None,
        partitioner=None,
        validate_shape=True,
        custom_getter=None
):
    """Wrapper over tf.get_variable().
    """

    with tf.device('/cpu:0'):
        var = tf.get_variable(
            name, shape=shape, dtype=dtype, initializer=initializer,
            regularizer=regularizer, trainable=trainable,
            collections=collections, caching_device=caching_device,
            partitioner=partitioner, validate_shape=validate_shape,
            custom_getter=custom_getter
        )
    return var


def _get_weights_wrapper(name, shape, wd=None):
    weights = _get_variable_wrapper(
        name=name, shape=shape, dtype=tf.float32,
        initializer=tf.truncated_normal_initializer(
            mean=0.0, stddev=0.05, dtype=tf.float32
        )
    )

    if wd is not None and wd > 0.0:
        weights_wd = tf.multiply(
            tf.nn.l2_loss(weights), wd, name=name + '/l2loss'
        )

        tf.add_to_collection('losses', weights_wd)

    return weights


def _get_biases_wrapper(name, shape):
    biases = _get_variable_wrapper(
        name=name, shape=shape, dtype=tf.float32,
        initializer=tf.constant_initializer(0.0)
    )

    return biases

# add an activation?
def _conv2d_wrapper(inputs, shape, strides, add_bias, name):
    """Wrapper over tf.nn.conv2d().
    """

    with tf.variable_scope(name) as scope:
        kernel = _get_weights_wrapper(
            name='weights', shape=shape, wd=0.0
        )
        output = tf.nn.conv2d(
            inputs, filter=kernel, strides=strides, padding='SAME', name='conv'
        )
        if add_bias:
            biases = _get_biases_wrapper(
                name='biases', shape=[shape[-1]]
            )
            output = tf.add(
                output, biases, name='biasAdd'
            )

    return output

In [3]:
def capsule_init(inputs, shape, strides, capsule_shape, name):
    """This convert a conv layer to a capsule primary layer.
    
    Args:
        inputs: [N, H, W, C], N: batch_size, ..
        shape: kernel shape, say [K, K, I, O]
        strides: kernel strides, 
        capsule_shape: [4, 4]
        
    Return:
        (pose, activation)
    """
    
    assert len(capsule_shape) == 2
    
    pose = []
    for h0 in xrange(capsule_shape[0]):
        units = []
        for h1 in xrange(capsule_shape[1]):
            units.append(_conv2d_wrapper(inputs, shape, strides, False, 'pose_unit_'+str(h0)+'_'+str(h1)))
        pose.append(tf.stack(units, axis=-1))
    pose = tf.stack(pose, axis=-1)
    
    activation = _conv2d_wrapper(inputs, shape, strides, False, 'activation'+str(h0)+'_'+str(h1))
    
    activation = tf.sigmoid(activation)
    
    return pose, activation
    

In [15]:
_conv2d_wrapper(aa, shape=[2, 2, 3, 5], strides=[1, 1, 1, 1], add_bias=False, name='test')

<tf.Tensor 'test/conv:0' shape=(1, 4, 4, 5) dtype=float32>

In [16]:
pose1, activation1 = capsule_init(aa, shape=[2, 2, 3, 5], strides=[1, 1, 1, 1], capsule_shape=[4, 4], name='primary')

In [17]:
pose1

<tf.Tensor 'stack_4:0' shape=(1, 4, 4, 5, 4, 4) dtype=float32>

In [18]:
activation1

<tf.Tensor 'Sigmoid:0' shape=(1, 4, 4, 5) dtype=float32>

In [19]:
sess.run(tf.global_variables_initializer())

In [20]:
pose1v, activation1v = sess.run([pose1, activation1])

In [21]:
pose11 = tf.concat([pose1, pose1, pose1], axis=0)
pose11

<tf.Tensor 'concat:0' shape=(3, 4, 4, 5, 4, 4) dtype=float32>

In [22]:
activation11 = tf.concat([activation1, activation1, activation1], axis=0)
activation11

<tf.Tensor 'concat_1:0' shape=(3, 4, 4, 5) dtype=float32>

In [12]:
w2 = tf.convert_to_tensor(
    np.arange(1*4*4*5*4*4).reshape((1, 4, 4, 5, 4, 4)), dtype=tf.float32
)
w2

<tf.Tensor 'Const_2:0' shape=(1, 4, 4, 5, 4, 4) dtype=float32>

In [13]:
w22 = tf.tile(w2, [3, 1, 1, 1, 1, 1])
w22

<tf.Tensor 'Tile:0' shape=(3, 4, 4, 5, 4, 4) dtype=float32>

In [25]:
zz = tf.matmul(pose11, w22)
zz

<tf.Tensor 'MatMul_2:0' shape=(3, 4, 4, 5, 4, 4) dtype=float32>

In [ ]:
z2 = tf.stack([zz, zz], axis=-3)
# all i into 1dim
z3 = tf.reshape(z2, [3, 80, 2, 4, 4])
# all h into 1dim
z4 = tf.reshape(z3, [3, 80, 2, 16])

z4

In [ ]:
rr = tf.constant(1/2, shape=(3, 80, 2), dtype=tf.float32)
rr

In [ ]:
activation14 = tf.reshape(activation11, [3, 80])
activation14

In [ ]:
# one iteration
tt = 1

# m_step
# this is one conv op, scan through with stride, to have 14x14 -> 6x6 or what ever
# R -> rr, [80, 2], num_i * num_c 
# a -> activation14, [3, 4, 4, 5] -> [3, 80]
# V -> z4, [3, 80, 2, 16]

# for cx in 0, 1, 
# cx==0
ri = rr[:, :, 0]
rr_prime = ri * activation14

mu_h = tf.reduce_sum(z4[:, :, 0, :] * tf.expand_dims(rr_prime, -1), axis=1) / tf.expand_dims(tf.reduce_sum(rr_prime, axis=1), -1)

sigma2_h = tf.reduce_sum(
    tf.square((z4[:,:,0,:] - tf.expand_dims(mu_h, axis=1))) * tf.expand_dims(rr_prime, -1), 
    axis=1
) / tf.expand_dims(tf.reduce_sum(rr_prime, axis=1), -1)

# beta_v should be a trainable_variable
beta_v = tf.constant(1, dtype=tf.float32)
cost_h = (beta_v + tf.log(sigma2_h)) * tf.expand_dims(tf.reduce_sum(rr_prime, axis=1), -1)

# lambda (ll) and beta_a should be trainable variables
ll = tf.constant(1, dtype=tf.float32)
beta_a = tf.constant(1, dtype=tf.float32)
a_c = tf.sigmoid(ll * (beta_a - tf.reduce_sum(cost_h, axis=1)))

# this is one value of c -> vectorize to all channels

#? for loop to all positions

In [ ]:
rr_prime = ri * activation14
tf.reduce_sum(rr_prime, axis=1)

In [ ]:
a_c

In [ ]:
def m_step():
    pass

def e_step():
    pass

def capsule_routing_em():
    pass

In [ ]:
def capsule_conv(inputs, shape):
    """This connect two capsule layers with conv EM routing.
    
    Args:
        inputs: (pose, activation):
            - pose: [N, H, W, C, CH, CW], CH, CW are capsule shape CH=4 x CW=4.
            - activation: [N, H, W, C]
        shape: as conv [H, W, I, O, CH, CW]
    """
    
    view = _get_weights_wrapper(
        name='weights', shape=shape, wd=0.0
    )
    
    pass

In [ ]:
def capsule_fc():
    """EM"""
    pass

In [ ]:
b = tf.constant(np.arange(13, 19, dtype=np.int32),
                shape=[3, 2])
z = tf.matmul(a, b)
zv = sess.run(z)

y = tf.tensordot(a, b, axes=[[2], [0]])
yv = sess.run(y)


w1 = tf.constant(
    np.arange(3 * 12).reshape(1, 1, 1, 3 * 12), dtype=tf.float32
)
w1

In [ ]:
bb = tf.matmul(aa, w1)

# 1 x 1 x 32 x 4 x 4

# how many W? 32

# pose
# [batch_size, 4, 4, 6, 6, 11]


In [ ]:
def caspule_primary(input, kernel=[2, 2], h=[4, 4], activation_fn=tf.sigmoid):
    pass # for loop then tf.stack
    # return pose [batch_size, h0, h1, h, w, d], activation [batch_size, h, w, d]

In [ ]:
def capsule_convolution():
    pass

In [ ]:
zv.shape

In [ ]:
yv.shape

In [ ]:
bv = sess.run(bb)

In [ ]:
from IPython.display import clear_output, Image, display, HTML

In [ ]:
# Helper functions for TF Graph visualization

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = tf.compat.as_bytes("<stripped %d bytes>"%size)
    return strip_def
  
def rename_nodes(graph_def, rename_func):
    res_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = res_def.node.add() 
        n.MergeFrom(n0)
        n.name = rename_func(n.name)
        for i, s in enumerate(n.input):
            n.input[i] = rename_func(s) if s[0]!='^' else '^'+rename_func(s[1:])
    return res_def
  
def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))
  
    iframe = """
        <iframe seamless style="width:800px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

In [ ]:
show_graph(graph_def=graph)